<a href="https://colab.research.google.com/github/jj0ng/TIL/blob/main/sentiment_analysis_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with Python
Main goal: sentiment analysis on some tweets   
Sub goal: get comfortable with common ML tools for python (throw them on your resume!)

<!-- 
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment?text=I+like+you.+I+love+you -->


In [ ]:
x = 10
y = x + 5

In [ ]:
print(y)

15


In [ ]:
word = "hello"

x + word

TypeError: ignored

# Day 1

### Set up the environment
Colab is hosting a workspace for us, and **we need to fill it with python tools**. Some of them are already added to the environment, ready to import into the notebook, but some need to be installed first. 

Colab is wonderful because you can run on google's servers, either gpu (for fast computation) or cpu (like your local computer). The only issue is since it's free, it occasionally kicks you off the gpu to make room for paying customers :( 

You can choose to work on google hosted servers or your own computer by clicking the dropdown where it says "RAM" in the top right corner.

In [ ]:
# '!' means these commands will execute on the command line, making changes outside of the notebook. 
# pip is a command to install packages in the workspace
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 20.6 MB/s eta 0:00:00


In [ ]:
# huggingface's tools for pretrained language models
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer

In [ ]:
# importing miscelaneaous packages 
# each module contains code like classes and functions, that we want to use

import numpy as np # fast manipulation of multidimensional arrays
from numpy import mean

from tqdm.notebook import tqdm as progress_bar # a little vizualization of how fast a loop is running
from scipy.special import softmax
import csv
from datetime import datetime
from matplotlib.dates import date2num

# more packages, tools for getting to google drive
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

You Try: Accessing the documentation

All these modules have associated documentation the author wrote for us. We can google the documentation, hover over a module, or literally type a question mark.

For instance, I could run `csv?` to learn what kind of a module csv is.

In the next cell, pick one of the modules we imported and learn something - ANYTHING - about it. We'll share.

In [ ]:
# you try

mean?

### Load Data to DataFrame
We will load some tweets from a shared google drive file, which will require us to sign into our google drive accounts. 
<!-- [backup link to shared file]() -->

<!-- # id = '1prY6_BgwYrUHCSJ0DYAB4KA7HcjTdDtU' -->

*key python package: pandas*   
[pandas documentation](https://pandas.pydata.org/docs/getting_started/index.html#getting-started). 

[visual cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)

[how to create a new dataframe](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)

In [ ]:
import pandas as pd # basically the excel of python

##### access data

We establish that this notebook is using our personal google account, and its not some random web scraper trying to look at data

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

I have a spreadsheet shared through google drive. The share link contains the id google uses to keep track of that file - I pass the link to GoogleDrive object and it goes and brings the file to our workspace.

In [ ]:
# I got this id from the link to the shared data
id = "10aCSo7OCVeuF7LSg4Rc1KijYICFYe_Wb"

# This step will move the file from Drive to the workspace
# it will take a little bit of time
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('twitter_data_full.xlsx')
print("Added file to workspace")

# This step will load the file from the workspace to our notebook session
df = pd.read_excel('twitter_data_full.xlsx')
print("Loaded file to notebook as a dataframe")

Added file to workspace
Loaded file to notebook as a dataframe


#### using pandas

In [ ]:
# df is now an object, with associated methods we can use
df.head(5) # lets look at the first five data samples
# you can even access the spreadsheet in colab... 

,Date,Full Text,Clean Text,Url,Emotion,Language,Country Code,Continent Code,Continent,Country,...,Interest,Mentioned Authors,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Retweets,Twitter Tweets,Twitter Verified,Reach (new),Twitter Likes
0,2022-10-01 23:40:00.000,"In Colorado Senate race, Michael Bennet still ...","in colorado senate race, michael bennet still ...",http://twitter.com/Prison_Health/statuses/1576...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,Shopping,NaN,19711.0,2715.0,0.0,0.0,441744.0,False,7325.0,40.006579
1,2022-10-01 23:27:28.000,COMING UP on @WRAL at 7:30pm: We sit down with...,coming up on at 7:30pm: we sit down with and...,http://twitter.com/BryanRAnderson/statuses/157...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,Politics,"@wral, @bohines, @wileynickel",3832.0,1103.0,2.0,4.0,17642.0,True,13263.0,40.006579
2,2022-10-01 23:16:38.000,Summaries of high-profile Supreme Court cases:...,summaries of high-profile supreme court cases:...,http://twitter.com/January20th49/statuses/1576...,Fear,en,USA,NORTH AMERICA,North America,United States of America,...,NaN,NaN,39.0,300.0,0.0,1.0,5523.0,False,0.0,40.006579
3,2022-10-01 23:05:12.000,Abortion Icon Emma Bonino Trounced in Italian ...,abortion icon emma bonino trounced in italian ...,http://twitter.com/UsBurning/statuses/15763475...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,NaN,NaN,360.0,34.0,0.0,0.0,23157.0,False,0.0,40.006579
4,2022-10-01 22:02:12.000,💥38 DAYS UNTIL #ELECTIONDAY MIDTERMS💥 WHAT R U...,💥38 days until midterms💥 what r u doing for d...,http://twitter.com/LeviFetterman/statuses/1576...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,Animals & Pets,NaN,33774.0,1702.0,1.0,10.0,49930.0,False,16039.0,40.006579


In [ ]:
# but we don't want to have to use the actual spreadsheet. pandas is better. 
df.columns

Index(['Date', 'Full Text', 'Clean Text', 'Url', 'Emotion', 'Language',
       'Country Code', 'Continent Code', 'Continent', 'Country', 'Region Code',
       'Region', 'City Code', 'Account Type', 'Author', 'City', 'Full Name',
       'Gender', 'Hashtags', 'Impact', 'Impressions', 'Interest',
       'Mentioned Authors', 'Twitter Followers', 'Twitter Following',
       'Twitter Reply Count', 'Twitter Retweets', 'Twitter Tweets',
       'Twitter Verified', 'Reach (new)', 'Twitter Likes'],
      dtype='object')

In [ ]:
len(df)

19057

In [ ]:
# look at one column
df['Region']

0                Hawaii
1        North Carolina
2                  Ohio
3               Georgia
4          Pennsylvania
              ...      
19052               NaN
19053          Arkansas
19054               NaN
19055      Pennsylvania
19056     Massachusetts
Name: Region, Length: 19057, dtype: object

In [ ]:
df['Region'].value_counts()

California              1763
Texas                   1759
District of Columbia    1492
New York                1442
Florida                 1163
Arizona                  605
Illinois                 538
Massachusetts            428
Pennsylvania             422
New Jersey               385
Georgia                  353
Virginia                 351
Ohio                     287
Washington               285
Maryland                 242
Colorado                 233
Michigan                 224
Tennessee                196
North Carolina           181
Nevada                   178
Missouri                 173
Minnesota                166
Oregon                   153
Wisconsin                146
Indiana                  120
Oklahoma                 109
New Hampshire            102
Alabama                   93
Iowa                      89
South Carolina            85
Kansas                    84
Kentucky                  82
Louisiana                 78
Connecticut               73
Utah          

In [ ]:
# group by a variable
df.groupby('Region').mean()

,Impact,Impressions,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Retweets,Twitter Tweets,Twitter Verified,Reach (new),Twitter Likes
Region,,,,,,,,,,
Alabama,15.165591,27843.430108,12845.215054,1818.612903,1.021505,3.623656,44340.881720,0.107527,4753.516129,40.231183
Alaska,11.794444,5929.666667,2785.166667,2786.722222,0.166667,0.500000,50482.333333,0.000000,1823.111111,28.893640
Arizona,22.413719,54976.122314,14933.547107,2925.957025,3.095868,8.494215,32413.793388,0.135537,9480.183471,39.178404
Arkansas,22.390566,76934.264151,22556.018868,2451.113208,18.679245,25.245283,40423.056604,0.132075,35314.622642,105.986470
California,20.906296,79488.508792,34098.574589,4654.990357,1.985820,7.391378,57661.403857,0.101531,8204.674419,37.806684
Colorado,18.880258,57742.896996,16028.901288,2957.939914,12.231760,18.540773,46598.377682,0.081545,23715.815451,30.215129
Connecticut,16.001370,10736.794521,4733.794521,1751.246575,0.726027,3.178082,55815.164384,0.054795,3603.342466,33.183309
Delaware,16.983333,15898.875000,4750.083333,1233.166667,0.750000,1.583333,92135.416667,0.041667,3810.583333,31.671875
District of Columbia,39.585188,352666.468499,176705.737936,2854.010054,7.266756,14.073056,71210.862601,0.476542,23988.177614,41.519168


In [ ]:
# sort values
df.sort_values(by='Twitter Likes', ascending=False)

,Date,Full Text,Clean Text,Url,Emotion,Language,Country Code,Continent Code,Continent,Country,...,Interest,Mentioned Authors,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Retweets,Twitter Tweets,Twitter Verified,Reach (new),Twitter Likes
15989,2022-10-27 02:02:07.000,"EVERYONE shows up, and we keep the House, add ...","everyone shows up, and we keep the house, add ...",http://twitter.com/duty2warn/statuses/15854517...,Disgust,en,USA,NORTH AMERICA,North America,United States of America,...,Beauty/Health & Fitness,NaN,370120.0,12121.0,1615.0,10070.0,51522.0,False,4333753.0,31307.0
17617,2022-11-03 15:39:43.000,I’m an immigration lawyer who represents refug...,i’m an immigration lawyer who represents refug...,http://twitter.com/mkolken/statuses/1588194219...,Fear,en,USA,NORTH AMERICA,North America,United States of America,...,"Animals & Pets, Family & Parenting",NaN,114892.0,4057.0,1095.0,4118.0,107978.0,True,2405101.0,21136.0
18423,2022-11-08 22:45:56.000,CBS Pennsylvania Exit Poll: Which ONE of these...,cbs pennsylvania exit poll: which one of these...,http://twitter.com/PpollingNumbers/statuses/15...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,Politics,NaN,232718.0,282.0,552.0,1493.0,13919.0,False,1118481.0,8511.0
15710,2022-10-03 22:17:40.000,It's not just that Dr. Oz killed 300 dogs in v...,it's not just that dr. oz killed 300 dogs in v...,http://twitter.com/QasimRashid/statuses/157706...,Disgust,en,USA,NORTH AMERICA,North America,United States of America,...,"Books, Politics, Family & Parenting",NaN,352995.0,334.0,110.0,2372.0,11381.0,True,674728.0,8493.0
15870,2022-10-27 23:22:16.000,"If Republicans were serious about the border, ...","if republicans were serious about the border, ...",http://twitter.com/mmpadellan/statuses/1585773...,Disgust,en,USA,NORTH AMERICA,North America,United States of America,...,"Books, Politics, Family & Parenting",NaN,1094654.0,45349.0,1674.0,2265.0,129504.0,True,2858902.0,7130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16591,2022-10-21 20:12:55.000,No.. they never have a plan .. even for health...,no.. they never have a plan .. even for health...,http://twitter.com/myguarddog47/statuses/15835...,Sadness,en,USA,NORTH AMERICA,North America,United States of America,...,NaN,NaN,18.0,42.0,0.0,0.0,6485.0,False,0.0,0.0
16594,2022-10-21 20:01:33.000,⬇️THE ONLY RECOURSE IS TO VOTE DEMOCRAT TO STO...,⬇️the only recourse is to vote democrat to sto...,http://twitter.com/EvAndee4ever/statuses/15835...,Sadness,en,USA,NORTH AMERICA,North America,United States of America,...,NaN,NaN,159.0,538.0,0.0,0.0,24261.0,False,0.0,0.0
16595,2022-10-21 19:51:53.000,Why is immigration such a big deal in the Texa...,why is immigration such a big deal in the texa...,http://twitter.com/GilbertBailon/statuses/1583...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,Books,"@nachoaguilar, @keranews",3050.0,587.0,0.0,0.0,38647.0,True,1963.0,0.0
16596,2022-10-21 19:26:30.000,"If your issue is crime, why vote for the party...","if your issue is crime, why vote for the party...",http://twitter.com/cynsinaspen/statuses/158354...,Anger,en,USA,NORTH AMERICA,North America,United States of America,...,"Business, Animals & Pets",NaN,294.0,1062.0,0.0,0.0,5739.0,False,0.0,0.0


In [ ]:
# replace missing data
df['Twitter Likes'].fillna(df['Twitter Likes'].mean(), inplace=True)

You try: apply one of the commands to a different column variable.


### Sentiment Classification
We will load up a pretrained model to our workspace. This model will first tokenize tweets, then map tweets to an embedding space where similar words are near to each other. Then the model will classify each tweet into "positive", "negative", or "neutral" data. 

This model was trained on hand-labeled tweets until it reach a high score on the training task, so we can trust it to be pretty good as long as we only use it on tweet-like data.

*key python package: transformers by Hugging Face*   
[our model](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest?text=Covid+cases+are+increasing+fast%21)

[how large language models work (specifically chapter 2 but all chapters are good)](https://www.pinecone.io/learn/sentence-embeddings/)


First, lets understand where our model is coming from

[Hugging Face model repository](https://huggingface.co/models)


In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

# preprocess
tokenizer = AutoTokenizer.from_pretrained(MODEL)
# you can choose from many tokenizing model options as well, scroll over the method

# load up model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# our sentiment analysis model is going to classify every tweet as one of these labels
labels = ["negative", "neutral", "positive"]

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model = model.to('cpu')

def print_scores(text):
  """a function to print out sentiment scores for one text sample in a pretty way, from the model author's demo"""
  print(text)
  
  #tokenize and encode
  encoded_input = tokenizer(text, return_tensors='pt')
  # get the sentiment classification
  output = model(**encoded_input)
  # transform data to correct format
  scores = output[0][0].detach().numpy()
  # apply softmax to get a probability distribution
  scores = softmax(scores)

  # print
  for i in range(scores.shape[0]):
    l = labels[i]
    s = scores[i]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")
  print()
  return 

#### Test a few sample sentences

Sentiment analysis has its limits. When it fails, it fails silently.

In [ ]:
# let's test a few sentences
print_scores("I ❤️ u")
print_scores("I love you")
print_scores("I love you but")
print_scores("I cannot love you")

I ❤️ u
1) negative 0.0064
2) neutral 0.0585
3) positive 0.935

I love you
1) negative 0.0181
2) neutral 0.1224
3) positive 0.8595

I love you but
1) negative 0.1123
2) neutral 0.4319
3) positive 0.4558

I cannot love you
1) negative 0.6917
2) neutral 0.2625
3) positive 0.0458



In [ ]:
# what about hard sentiment questions?
# constrast
print_scores("The weather was terrible, but the hike was amazing!")
# sarcasm
print_scores("I’m so happy the plane is delayed.")

The weather was terrible, but the hike was amazing!
1) negative 0.1333
2) neutral 0.119
3) positive 0.7476

I’m so happy the plane is delayed.
1) negative 0.0592
2) neutral 0.1096
3) positive 0.8312



You try to use `print_scores`, with your own sentence. Can you think of anything that confuses the model?

#### Apply roberta to tweets
Group our tweets into batches and get many scores at a time. 
I'm going to demo this - I wouldn't recommend changing the code at this point

*relevant python package: pytorch*

If we put out model on "CUDA" the notebook will process many tweets in at the same time. Let google's servers do the hard work...

In [ ]:
# deep learning toolkit
from torch.utils.data import DataLoader
from torch.nn import Softmax
import torch

In [ ]:
CUDA = torch.cuda.is_available()
print("Using gpu:", CUDA)

Using gpu: True


In [ ]:
# only cuda if its available
processor = 'cuda'
# processor = 'cpu'


We will make a `DataLoader` object that groups tweets into small batches, so that we can process them simultaneously.

The dataloader might seem arbitrary now but becomes useful once we want to control more factors of the dataset.

In [ ]:
# making some pytorch variables to assist us
batch_size = 32
dl = DataLoader(df['Clean Text'], batch_size=batch_size)
print("Number of Batches", len(dl))

Number of Batches 596


In [ ]:
# lets run it! It should take about a minute or so

# this line makes an empty dataframe to hold the scores we are about to calculate
sentiment_df = pd.DataFrame(columns=labels)

# this moves the model to a out chosen processor
model = model.to(processor)

for text_batch in progress_bar(dl): # we loop through every batch in our dataset

  encoded_input = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True) # turn each tweet string into tokens
  encoded_input.to(processor) # input must be on the same processor as the model

  output = model(**encoded_input) # apply the model

  embeddings = output[0].detach().cpu().numpy() # gross rearrangment of the data type, a messy intermediate step

  # append new embeddings to current dataframe through pandas manipulations
  embed_df = pd.DataFrame(embeddings, columns=labels)
  sentiment_df = pd.concat((sentiment_df, embed_df), axis=0, ignore_index=True)
  
# convert embeddings (vectors) to a probability distribution
sentiment_df = sentiment_df.apply(softmax, axis=1) 

  0%|          | 0/596 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
sentiment_df

,negative,neutral,positive
0,0.019214,0.897980,0.082806
1,0.009155,0.954184,0.036661
2,0.019073,0.949389,0.031538
3,0.367401,0.598292,0.034307
4,0.062576,0.763785,0.173639
...,...,...,...
19052,0.780095,0.205532,0.014372
19053,0.142690,0.676964,0.180346
19054,0.039305,0.935626,0.025069
19055,0.722581,0.259136,0.018283


## Analyze (with pandas)

In [ ]:
# Transform scores to labels
def get_class(row):
  max_idx = np.argmax(row)
  return labels[max_idx]

sentiment_df["predicted_class"] = sentiment_df.apply(get_class, axis=1)

In [ ]:
sentiment_df

,negative,neutral,positive,predicted_class
0,0.019214,0.897980,0.082806,neutral
1,0.009155,0.954184,0.036661,neutral
2,0.019073,0.949389,0.031538,neutral
3,0.367401,0.598292,0.034307,neutral
4,0.062576,0.763785,0.173639,neutral
...,...,...,...,...
19052,0.780095,0.205532,0.014372,negative
19053,0.142690,0.676964,0.180346,neutral
19054,0.039305,0.935626,0.025069,neutral
19055,0.722581,0.259136,0.018283,negative


In [ ]:
df = pd.concat((df, sentiment_df), axis=1)

In [ ]:
df.head()

,Date,Full Text,Clean Text,Url,Emotion,Language,Country Code,Continent Code,Continent,Country,...,Twitter Reply Count,Twitter Retweets,Twitter Tweets,Twitter Verified,Reach (new),Twitter Likes,negative,neutral,positive,predicted_class
0,2022-10-01 23:40:00.000,"In Colorado Senate race, Michael Bennet still ...","in colorado senate race, michael bennet still ...",http://twitter.com/Prison_Health/statuses/1576...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,0.0,0.0,441744.0,False,7325.0,40.006579,0.019214,0.897980,0.082806,neutral
1,2022-10-01 23:27:28.000,COMING UP on @WRAL at 7:30pm: We sit down with...,coming up on at 7:30pm: we sit down with and...,http://twitter.com/BryanRAnderson/statuses/157...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,2.0,4.0,17642.0,True,13263.0,40.006579,0.009155,0.954184,0.036661,neutral
2,2022-10-01 23:16:38.000,Summaries of high-profile Supreme Court cases:...,summaries of high-profile supreme court cases:...,http://twitter.com/January20th49/statuses/1576...,Fear,en,USA,NORTH AMERICA,North America,United States of America,...,0.0,1.0,5523.0,False,0.0,40.006579,0.019073,0.949389,0.031538,neutral
3,2022-10-01 23:05:12.000,Abortion Icon Emma Bonino Trounced in Italian ...,abortion icon emma bonino trounced in italian ...,http://twitter.com/UsBurning/statuses/15763475...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,0.0,0.0,23157.0,False,0.0,40.006579,0.367401,0.598292,0.034307,neutral
4,2022-10-01 22:02:12.000,💥38 DAYS UNTIL #ELECTIONDAY MIDTERMS💥 WHAT R U...,💥38 days until midterms💥 what r u doing for d...,http://twitter.com/LeviFetterman/statuses/1576...,NaN,en,USA,NORTH AMERICA,North America,United States of America,...,1.0,10.0,49930.0,False,16039.0,40.006579,0.062576,0.763785,0.173639,neutral


In [ ]:
df.columns

Index(['Date', 'Full Text', 'Clean Text', 'Url', 'Emotion', 'Language',
       'Country Code', 'Continent Code', 'Continent', 'Country', 'Region Code',
       'Region', 'City Code', 'Account Type', 'Author', 'City', 'Full Name',
       'Gender', 'Hashtags', 'Impact', 'Impressions', 'Interest',
       'Mentioned Authors', 'Twitter Followers', 'Twitter Following',
       'Twitter Reply Count', 'Twitter Retweets', 'Twitter Tweets',
       'Twitter Verified', 'Reach (new)', 'Twitter Likes', 'negative',
       'neutral', 'positive', 'predicted_class'],
      dtype='object')

In [ ]:
df['predicted_class'].value_counts()

negative    11048
neutral      6445
positive     1564
Name: predicted_class, dtype: int64

In [ ]:
df.groupby('Twitter Verified').mean()

,Impact,Impressions,Twitter Followers,Twitter Following,Twitter Reply Count,Twitter Retweets,Twitter Tweets,Reach (new),Twitter Likes,negative,neutral,positive
Twitter Verified,,,,,,,,,,,,
False,17.161753,30970.247050,5307.548303,3398.410999,1.188527,5.330028,53371.244738,4966.400353,35.723064,0.548275,0.348158,0.103566
True,54.780527,653650.754868,456537.972890,3473.822833,16.134402,32.922489,82819.854525,47550.613975,66.891810,0.322997,0.534736,0.142267


## Save your work

In [ ]:
# let's save it
# this will only save to worksplace and then you have to download it to your local computer or upload to drive
df.to_excel("roberta_sentiment_scores.xlsx")

# What to expect for Day 2
In the next two weeks, you can explore this topic and try it on a new problem. Next meeting, we will share our findings, debug any issues as a group, and demo a couple more python tools.

Your goal is to change one or two things about our model pipeline. Here are some ideas:


**Easiest**:
Try to run the same code on different data

**More effort**:
Try to use a different tokenizer or sentiment analysis model from hugginface. For example, use a model that has been trained on multiple languages or Reddit data. 

**Advanced**:
Try another classification task instead of sentiment, like labeling emotions or sentence subjectivity.


**Easy places to get data?**  
[kaggle](https://www.kaggle.com/datasets)  
["25 Twitter Datasets for Natural Language Processing"](https://imerit.net/blog/top-25-twitter-datasets-for-natural-language-processing-and-machine-learning-all-pbm/)  

# Appendix

## If you get an error

1) Check that spelling and case exactly matches what you want to reference. `Variable` and `variable` are not equivalent.  
2) Are strings wrapped in quotations, like `"string"`? Did you mix up `[` and `(`?  
3) Put a screenshot of the cell and the error message in the chat, or copy and paste the code.  
4) Try to change one thing at a time so you learn what fixes the code.   
5) Remember the fix so you can help out the next person who gets that error.

## How to clean text data

Gameplan: define a function that cleans one tweet. Then apply this function to every tweet in our dataframe.

Regex is a shorthand for writing logical statements to match strings - we are just copying and pasting the regex today, its not important to learn all the shorthand yet. But if you are interested:

[using regex to remove hashtags](https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/)   
[regex cheatsheet](https://www.interviewbit.com/regex-cheat-sheet/)


*key python package: regex*

In [ ]:
import re # search through and clean text

In [ ]:
# there is a problem with the data oh no!
print(df['Full Text'][0])
# Most language models probably don't know what the hell 'https://t.co/F5ak34HrCE' is

In Colorado Senate race, Michael Bennet still fights for child tax credit and immigration reform https://t.co/F5ak34HrCE


In [ ]:
# tabs are important in python
# running this cell defines the function, does not run the function

def clean(tweet):
  # remove uppercase letters
  # remove mentions
  # remove hashtags
  # remove links

  tweet = tweet.lower()

  tweet = re.sub("@[A-Za-z0-9_]+", "", tweet)
  tweet = re.sub("#[A-Za-z0-9_]+", "", tweet)
  tweet = re.sub(r"http\S+", "", tweet)

  return tweet

In [ ]:
df['my_cleaned_text'] = df['Full Text'].apply(clean)

In [ ]:
print(df['my_cleaned_text'][0])

in colorado senate race, michael bennet still fights for child tax credit and immigration reform 
